Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
a = np.arange(36).reshape(4,3,3)
print(a)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]

 [[27 28 29]
  [30 31 32]
  [33 34 35]]]


In [3]:
b = a.reshape(4,9)
print(b)

[[ 0  1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16 17]
 [18 19 20 21 22 23 24 25 26]
 [27 28 29 30 31 32 33 34 35]]


In [4]:
pickle_file = 'notMNIST_cleaned.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (194661, 28, 28) (194661,)
Validation set (9967, 28, 28) (9967,)
Test set (9967, 28, 28) (9967,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

In [6]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (194661, 784) (194661, 10)
Validation set (9967, 784) (9967, 10)
Test set (9967, 784) (9967, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [7]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [8]:
num_steps = 801

def accuracy(predictions, labels, name='accuracy'):
    correct_predictions = tf.equal(tf.argmax(predictions,1), tf.argmax(labels,1))
    prediction_accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name=name)
    return prediction_accuracy

In [9]:
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))

Initialized
Loss at step 0: 18.443838
Loss at step 100: 2.321529
Loss at step 200: 1.890155
Loss at step 300: 1.655883
Loss at step 400: 1.497687
Loss at step 500: 1.378948
Loss at step 600: 1.284428
Loss at step 700: 1.206504
Loss at step 800: 1.140868


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `sesion.run()`.

In [10]:
batch_size = 128

#TODO renommer les variables du modèle !
#les E/S du modèle doivent avoir des noms neutres : elles peuvent tout accepter (train / valid /test)
#tf_train_dataset / tf_train_labels / train_prediction
#virer les préfixes "tf"...... et mettre un préfixe aux constantes et valeurs numpy

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size), name='train_dataset')
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels), name='train_labels')
    tf_valid_dataset = tf.constant(valid_dataset, name='valid_dataset')
    tf_valid_labels = tf.constant(valid_labels, name='valid_labels')
    tf_test_dataset = tf.constant(test_dataset, name='test_dataset')
    tf_test_labels = tf.constant(test_labels, name='test_labels')

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]), name='weights_1')
    biases = tf.Variable(tf.zeros([num_labels]), name='labels_1')

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels), name='loss')

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data. !!!NULNULNUL!!!
    train_prediction = tf.nn.softmax(logits, name='train_prediction')
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases, name='valid_prediction')
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases, name='test_prediction')
    
    # compute the accuracy of the model !!!NULNULNUL!!!
    train_accuracy = accuracy(train_prediction, tf_train_labels, name='train_accuracy')
    valid_accuracy = accuracy(valid_prediction, tf_valid_labels, name='valid_accuracy')
    test_accuracy = accuracy(test_prediction, tf_test_labels, name='test_accuracy')

Let's run it:

In [11]:
num_steps = 6001

with tf.Session(graph=graph) as session:
    
    # log the quality of the model
    train_accuracy_log = tf.scalar_summary("train accuracy", train_accuracy)
    valid_accuracy_log = tf.scalar_summary("valid accuracy", valid_accuracy)
    test_accuracy_log = tf.scalar_summary("test accuracy", test_accuracy)
    loss_log = tf.scalar_summary("loss", loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter("/tmp/udacity_logs", session.graph_def)
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        if (step % 500 == 0):
            #evaluate on the validation & test sets
            feed_dict = {tf_train_dataset : train_dataset, tf_train_labels : train_labels}
            l, s = session.run([loss, merged], feed_dict=feed_dict)
            writer.add_summary(s, step)
            print("==> step %d" % step)
            print("Validation loss: %f" % l)
        else:
            # Generate a minibatch.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            session.run(optimizer, feed_dict=feed_dict)

Initialized
==> step 0
Validation loss: 20.211836
==> step 500
Validation loss: 1.624196
==> step 1000
Validation loss: 1.324729
==> step 1500
Validation loss: 1.166878
==> step 2000
Validation loss: 1.054569
==> step 2500
Validation loss: 1.001581
==> step 3000
Validation loss: 0.911340
==> step 3500
Validation loss: 0.886249
==> step 4000
Validation loss: 0.864132
==> step 4500
Validation loss: 0.848066
==> step 5000
Validation loss: 0.806943
==> step 5500
Validation loss: 0.789622
==> step 6000
Validation loss: 0.770927


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [12]:
#

class Network(object):
    
    # PUBLIC ===============================================================
    
    def __init__(self, input_dataset):
        self.batch_size = int(input_dataset.get_shape()[0])
        self._vars = []
        self._vardict = {}
        self._add('input', input_dataset)
        self._setup()

    def get_output(self):
        return self._vars[-1][1]
    
    # PRIVY ================================================================

    def _setup(self):
        raise NotImplementedError('Must be subclassed.')

    def _get_unique_name_(self, prefix):        
        id = sum(t.startswith(prefix) for t,_ in self._vars)+1
        return '%s_%d'%(prefix, id)

    def _add(self, name, var):
        self._vars.append((name, var))
        self._vardict[name] = var
    
    def _linear(self, in_size, out_size, name=None):
        name = name or self._get_unique_name_('linear')
        weigths = tf.Variable(tf.truncated_normal([in_size, out_size]))
        biases = tf.Variable(tf.zeros([out_size]))
        logits = tf.matmul(self.get_output(), weights) + biases
        self._add(name, logits)
        return self
        
    def _relu(self, name=None):
        name = name or self._get_unique_name_('relu')
        relu = tf.nn.relu(self.get_output(), name=name)
        self._add(name, relu)

In [13]:
batch_size = 128

input_size = image_size * image_size
hidden_size = 90

class TwinHead(Network):
    def _setup(self):
        (self._linear(input_size, hidden_size, name='linear_1')
             ._relu(name='relu_1')
             ._linear(hidden_size, num_labels, name='linear_2'))

    def y(self):
        return [self.vardict['relu_1'], self.vardict['linear_2']]

---
Todo
-------

~~1) ajouter la possibilité de lancer une inférence (et donc transformer l'input en variable, qui peut être chargé soit avec les données d'entrainement soit avec celles de validation)~~

2) dissocier les parties du modèles :
- logique
- initialisation
- import des valeurs

~~3) historiser le coût et la précision du modèle~~

4) transférer ces fonctions dans le programme "neural art"

5) renommer toutes les variables :
- retirer le préfixe "tf"
- donner des noms génériques aux entrées du modèle
- évaluer les prédictions avec un dictionnaire test / train / valid différent à chaque fois 

---

In [18]:
batch_size = 128

input_size = image_size * image_size
hidden_size = 90
output_size = num_labels

num_steps = 6000

graph = tf.Graph()
with graph.as_default():

    # Generic placeholders : it can either run with ttrain / test / valid data
    input_data = tf.placeholder(tf.float32, shape=(None, input_size), name='input')
    output_data = tf.placeholder(tf.float32, shape=(None, output_size), name='output')
    
    # convert the validation / test datasets in tf nodes
    with tf.variable_scope('datasets') as scope:
        tf_valid_dataset = tf.constant(valid_dataset, name='valid_dataset')
        tf_valid_labels = tf.constant(valid_labels, name='valid_labels')
        tf_test_dataset = tf.constant(test_dataset, name='test_dataset')
        tf_test_labels = tf.constant(test_labels, name='test_labels')

    # First layer
    with tf.variable_scope('first_layer') as scope:
        layer_1_weights = tf.Variable(tf.truncated_normal([input_size, hidden_size]), name='layer_1_weights')
        layer_1_biases = tf.Variable(tf.zeros([hidden_size]), name='layer_1_biases')
        layer_1_logits = tf.matmul(input_data, layer_1_weights) + layer_1_biases
        layer_1_relu = tf.nn.relu(layer_1_logits, name='layer_1_relu')
    
    # Second layer
    with tf.variable_scope('second_layer') as scope:
        layer_2_weights = tf.Variable(tf.truncated_normal([hidden_size, output_size]), name='layer_2_weights')
        layer_2_biases = tf.Variable(tf.zeros([output_size]), name='layer_2_biases')
        layer_2_logits = tf.matmul(layer_1_relu, layer_2_weights) + layer_2_biases
    
    # Output = prediction
    with tf.variable_scope('prediction') as scope:
        model_output = tf.nn.softmax(layer_2_logits, name='model_prediction')
        model_accuracy = accuracy(model_output, output_data, name='model_accuracy')
    
    # Loss
    with tf.variable_scope('loss') as scope:
        accuracy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2_logits, output_data), name='loss')
        regularization_loss = tf.nn.l2_loss(layer_1_weights) + tf.nn.l2_loss(layer_1_biases) + \
                              tf.nn.l2_loss(layer_2_weights) + tf.nn.l2_loss(layer_2_biases)
        loss = accuracy_loss + 0.001 * regularization_loss
    
    # Optimizer with dynamic learning rate
    with tf.variable_scope('optimizer') as scope:
        global_step = tf.Variable(0, trainable=False)
        learning_rate = tf.train.exponential_decay(0.5, global_step, num_steps * batch_size, 0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [19]:
with tf.Session(graph=graph, config=tf.ConfigProto(allow_soft_placement=True)) as session:
    
    # log the quality of the model
    train_accuracy_log = tf.scalar_summary("train accuracy", model_accuracy)
    valid_accuracy_log = tf.scalar_summary("validation accuracy", model_accuracy)
    test_accuracy_log = tf.scalar_summary("test accuracy", model_accuracy)
    loss_log = tf.scalar_summary("loss", loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs
    writer = tf.train.SummaryWriter("/tmp/udacity_logs", session.graph_def)
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(1,num_steps+1):
        if (step % 100 == 0):
            print("==> step %d" % step)
            
            #evaluate on the train set
            feed_dict = {input_data : train_dataset, output_data : train_labels}
            acc, acc_str = session.run([model_accuracy, train_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Train set accuracy: %f" % acc)
            
            #evaluate on the validation set
            feed_dict = {input_data : valid_dataset, output_data : valid_labels}
            acc, acc_str = session.run([model_accuracy, valid_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Validation set accuracy: %f" % acc)
            
            #evaluate on the test set
            feed_dict = {input_data : test_dataset, output_data : test_labels}
            acc, acc_str = session.run([model_accuracy, test_accuracy_log], feed_dict=feed_dict)
            writer.add_summary(acc_str, step)
            print("Test set accuracy: %f" % acc)
        else:
            # Generate a minibatch.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {input_data : batch_data, output_data : batch_labels}
            session.run(optimizer, feed_dict=feed_dict)

Initialized
==> step 100
Train set accuracy: 0.692743
Validation set accuracy: 0.767232
Test set accuracy: 0.767232
==> step 200
Train set accuracy: 0.665249
Validation set accuracy: 0.739240
Test set accuracy: 0.739240
==> step 300
Train set accuracy: 0.719420
Validation set accuracy: 0.780375
Test set accuracy: 0.780375
==> step 400
Train set accuracy: 0.737924
Validation set accuracy: 0.812080
Test set accuracy: 0.812080
==> step 500
Train set accuracy: 0.750721
Validation set accuracy: 0.821912
Test set accuracy: 0.821912
==> step 600
Train set accuracy: 0.760157
Validation set accuracy: 0.833250
Test set accuracy: 0.833250
==> step 700
Train set accuracy: 0.770493
Validation set accuracy: 0.845189
Test set accuracy: 0.845189
==> step 800
Train set accuracy: 0.771038
Validation set accuracy: 0.849102
Test set accuracy: 0.849102
==> step 900
Train set accuracy: 0.783706
Validation set accuracy: 0.842279
Test set accuracy: 0.842279
==> step 1000
Train set accuracy: 0.787549
Validatio

In [25]:
train_plot = np.genfromtxt('/home/mcm/Documents/Workspace/hacks/ai/udacity/plots/train_accuracy.csv', delimiter=',',
                           skip_header=1, names=['Wall time','Step','Value'])
valid_plot = np.genfromtxt('/home/mcm/Documents/Workspace/hacks/ai/udacity/plots/valid_accuracy.csv', delimiter=',',
                           skip_header=1, names=['Wall time','Step','Value'])
test_plot = np.genfromtxt('/home/mcm/Documents/Workspace/hacks/ai/udacity/plots/test_accuracy.csv', delimiter=',',
                           skip_header=1, names=['Wall time','Step','Value'])

In [28]:
plt.plot(train_plot['Step'], train_plot['Value'], 'r--')
plt.plot(valid_plot['Step'], valid_plot['Value'], 'bo')
plt.plot(test_plot['Step'], test_plot['Value'], 'g^')
plt.show()